In [1]:
import matplotlib as plt 
import numpy as np 
from matplotlib import colors
from PIL import Image, ImageDraw
import random
import pandas as pd
from collections import defaultdict
import random

In [2]:
def createList(r1, r2):#option creation for random edge addition
        if (r1 == r2):
            return r1
        else:
            res = []
            while(r1 != r2):
                res.append(r1)
                r1 += 1
                if (r1 > 50):
                    r1 = r1-50
            res.append(r2)
        return res 

In [3]:
class Graph():
    
    def __init__(self):
        
        #create graph
        self.graph = {}
        for i in range(1,51):
            if i-1 == 0:
                self.graph[i] = [2, 50]
            elif i+1 == 51:
                self.graph[i] = [1, 49]
            else:
                self.graph[i] = [i+1, i-1]
        
        #create list of nodes
        self.nodes = createList(1, 50)
                
    def addEdge(self):
        boo1 = True
        while boo1:
            if len(self.nodes) == 0:
                boo1 = False
                break
            val = random.choice(self.nodes)
            self.nodes.remove(val)
            valstart = val-5
            valend = val+5
            if (valstart < 1):
                valstart = valstart+50
            if (valend > 50):
                valend = valend-50
            val_nodes = createList(valstart, valend)
            if(val == 1): #deleting adjacent options so that double edges arent made
                val_nodes.remove(50)
            elif(val == 50):
                val_nodes.remove(1)
            else: 
                val_nodes.remove(val-1)
                val_nodes.remove(val+1)
            boo2 = False
            while len(val_nodes) > 0:
                newnode = random.choice(val_nodes)
                if newnode in self.nodes:
                    self.nodes.remove(newnode)
                    self.graph[val].append(newnode)
                    self.graph[newnode].append(val)
                    boo2 = True
                    break
                else:
                    val_nodes.remove(newnode)
                    
            if boo2:
                break
        if boo1 == False:
            return False
        else:
            return True

In [4]:
#start position, end position, backwards path to take
#returns path
def BFS(start, end, graph):
 
    # a queue of the vertices whose path is to be scanned
    # boolean array for checking if node has been visited at least once or not
    # all moves in path set to -1
    queue = []
    visited = [False for i in range(51)]
    back_path = [-1 for i in range(51)]
    
    # start is visited and added to the queue
    visited[start] = True
    queue.append(start)

    # BFS algorithm - look at all neighbors first
    while (len(queue) != 0):
        u = queue[0]
        queue.pop(0)
        for i in range(len(graph[u])):
            if (visited[graph[u][i]] == False):
                visited[graph[u][i]] = True
                back_path[graph[u][i]] = u
                queue.append(graph[u][i])

                # We stop BFS when we find
                # destination.
                if (graph[u][i] == end):
                    path = []
                    crawl = end
                    path.append(crawl)
                    
                    #reversing back_path and removing -1 from nodes that are not in the path to get full path
                    while (back_path[crawl] != -1):
                        path.insert(0, back_path[crawl])
                        crawl = back_path[crawl]
                    return path

    return [end]

In [5]:
class Predator():
    
    def __init__(self, agentloc):
        self.pos = random.randint(1,50)
        #making sure that predator doesn't start where the agent is
        while self.pos == agentloc:
            self.pos = random.randint(1,50)
        
    def pred_move(self, g, agentpos):
        
        #getting all next possible moves of predator
        neighbors = g.graph[self.pos]
        path = []
        shortest_path = []
        path_length = 50
        #finding shortest path to agent using BFS
        for neighbor in neighbors:
            path = BFS(neighbor, agentpos, g.graph)
            
            #saving shortest path(s)
            if(len(path) <= path_length):
                if(len(path) == path_length):
                    shortest_path.append(path)
                else:
                    shortest_path = []
                    shortest_path.append(path)
                    path_length = len(path)
        
        #randomly picking path of the shortest paths and moving to next neighbor
        new_path = random.randint(0,(len(shortest_path)-1))
        self.pos = shortest_path[new_path][0]
        return self


In [6]:
class Prey():
    
    def __init__(self, agentloc):
        self.pos = random.randint(1,50)
        #making sure prey doesn't start where agent is
        while self.pos == agentloc:
            self.pos = random.randint(1,50)
        
    def prey_move(self, g):
        prey_loc_neighbours = [self.pos]
        for i in g.graph[self.pos]:
            prey_loc_neighbours.append(i)
        #randomly choose neighbor to move to (or staying put)
        self.pos = random.choice(prey_loc_neighbours)
        return self

In [7]:
def Agent1(g):
    agent_pos = random.randint(1,50) #initializing agent
    predator = Predator(agent_pos) #initializing predator in location where agent doesnt exist
    prey = Prey(agent_pos) #initializing predator in location where agent doesnt exist
    fail = 0 #fail category initialization
    steps = 0 #to count steps taken while agent travels graph
    path = [] #path that agent Takes
    path.append(agent_pos) #start with agent location
    
    while (fail == 0 and steps <= 100):
        
        #print("Positions: " + str(agent_pos) + ", " + str(predator.pos) + ", "+ str(prey.pos))
        # getting all possible next positions of agent 1
        neighbors = g.graph[agent_pos]
        #neighbors.append(agent_pos)
        # moves = list of tuples - each path has a tuple with distance to prey, a distance to predator, and the neighbor they would move to for that path
        #keeping separate moves lists for each of dr cowans conditions
        moves_1 = []
        moves_2 = []
        moves_3 = []
        moves_4 = []
        moves_5 = []
        moves_6 = []
        path_to_prey = [] #initializing path from agents current position to prey
        path_to_predator = [] #initializing path from agents current position to predator
        curr_to_pred = len(BFS(agent_pos, predator.pos, g.graph)) #lenth of path from agents current position to predator
        curr_to_prey = len(BFS(agent_pos, prey.pos, g.graph)) #path from agents current position to prey
        
        for neighbor in neighbors:
            
            # shortest path from neighbor to prey and shortest path from neighbor to predator
            path_to_prey = BFS(neighbor, prey.pos, g.graph)
            path_to_predator = BFS(neighbor, predator.pos, g.graph)
            
            #print("Prey and Predator Moves")
            #print(path_to_prey)
            #print(path_to_predator)
            if (len(path_to_prey) < curr_to_prey and len(path_to_predator) > curr_to_pred): #agent 1 condition 1
                tup = (len(path_to_prey), len(path_to_predator), neighbor)
                moves_1.append(tup)
            
            elif (len(path_to_prey) < curr_to_prey and len(path_to_predator) == curr_to_pred): #agent 1 condition 2
                tup = (len(path_to_prey), len(path_to_predator), neighbor)
                moves_2.append(tup)

            elif (len(path_to_prey) == curr_to_prey and len(path_to_predator) > curr_to_pred): #agent 1 condition 3
                tup = (len(path_to_prey), len(path_to_predator), neighbor)
                moves_3.append(tup)

            elif (len(path_to_prey) == curr_to_prey and len(path_to_predator) == curr_to_pred): #agent 1 condition 4
                tup = (len(path_to_prey), len(path_to_predator), neighbor)
                moves_4.append(tup)

            elif (len(path_to_predator) > curr_to_pred): #agent 1 condition 5
                tup = (len(path_to_prey), len(path_to_predator), neighbor)
                moves_5.append(tup)

            elif (len(path_to_predator) == curr_to_pred): #agent 1 condition 6
                tup = (len(path_to_prey), len(path_to_predator), neighbor)
                moves_6.append(tup)          

        
        #print()
        if(len(moves_1) > 0): #if there are moves that satisfy condition 1
            moves_sorted = sorted(moves_1, key=lambda t: (t[0], -t[1])) #sorting by distance to prey
            #print(moves_sorted)
            move_choice=[]
            for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]):
                    move_choice.append(move)
            neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
            agent_pos = neighbour_choice[2]

        elif(len(moves_2) > 0): #if no moves satisfying condition 1, are there are moves that satisfy condition 2?
            moves_sorted = sorted(moves_2, key=lambda t: (t[0], -t[1])) #sorting by distance to prey
            #print(moves_sorted)
            move_choice=[]
            for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]):
                    move_choice.append(move)
            neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
            agent_pos = neighbour_choice[2]

        #so on and so forth from here on according to dr cowans condition

        elif(len(moves_3) > 0):
            moves_sorted = sorted(moves_3, key=lambda t: (t[0], -t[1]))
            #print(moves_sorted)
            move_choice=[]
            for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]):
                    move_choice.append(move)
            neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
            agent_pos = neighbour_choice[2]

        elif(len(moves_4) > 0):
            moves_sorted = sorted(moves_4, key=lambda t: (t[0], -t[1]))
            #print(moves_sorted)
            move_choice=[]
            for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]):
                    move_choice.append(move)
            neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
            agent_pos = neighbour_choice[2]

        elif(len(moves_5) > 0):
            moves_sorted = sorted(moves_5, key=lambda t: (-t[1],t[0]))
            #print(moves_sorted)
            move_choice=[]
            for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]):
                    move_choice.append(move)
            neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
            agent_pos = neighbour_choice[2]

        elif(len(moves_6) > 0):
            moves_sorted = sorted(moves_6, key=lambda t: (-t[1], t[0]))
            #print(moves_sorted)
            move_choice=[]
            for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]):
                    move_choice.append(move)
            neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
            agent_pos = neighbour_choice[2]
        
        #if none of the lists have a move available, means the agent should sit still and pray
        #if none of the if else cases were executed, the same agent  position will get appended again
        
        path.append(agent_pos)

        steps += 1

        if (agent_pos == prey.pos): #if agent has already moved to location of prey
            break

        #move predator and prey
        prey.prey_move(g)

        if (prey.pos == agent_pos): #if prey luckily moves to location of agent
            break

        predator.pred_move(g, agent_pos)

        if(predator.pos == agent_pos): #if predator catches agent
            fail = 1
            break
        
        
    #fail if timeout error    
    if(steps > 5000):
        fail = 2     
    return fail, path

In [8]:
wins = 0
losses = 0
timeout = 0
times = 0
win_dist = []
loss_dist = []

while times < 3000:
    g = Graph()
    graph = g.addEdge()
    while graph:
        graph = g.addEdge()
    res, path = Agent1(g)
    
    if(res == 0):
        wins += 1
        win_dist.append(len(path))
    elif(res == 1):
        losses += 1
        loss_dist.append(len(path))
    else:
        timeout += 1
    times += 1
    
print('Number of wins: ' + str(wins))
print("Max Steps Taken During a Win: " + str(max(win_dist)-1))
print("Average Steps Taken During a Win: " + str((sum(win_dist)/wins)-1))
print('Number of losses: ' + str(losses))
print("Max Steps Taken During a Loss: " + str(max(loss_dist)-1))
print("Average Steps Taken During a Loss: " + str((sum(loss_dist)/losses)-1))
print('Number of timeouts: ' + str(timeout))
print("Success Rate: " + str((wins/times)))

Number of wins: 2636
Max Steps Taken During a Win: 63
Average Steps Taken During a Win: 11.492033383915023
Number of losses: 364
Max Steps Taken During a Loss: 41
Average Steps Taken During a Loss: 10.45054945054945
Number of timeouts: 0
Success Rate: 0.8786666666666667


In [9]:
def Agent2(g):
    agent_pos = random.randint(1,50) #initializing agent
    predator = Predator(agent_pos) #initializing predator in location where agent doesnt exist
    prey = Prey(agent_pos) #initializing predator in location where agent doesnt exist
    fail = 0 #fail category initialization
    steps = 0 #to count steps taken while agent travels graph
    path = [] #path that agent Takes
    path.append(agent_pos) #start with agent location

    while(fail==0 and steps <= 100):
        neighbours = g.graph[agent_pos] #getting neighbours
        amazing_moves = [] #keeping list of moves that only satisfy Agent 1s first condition
        path_to_prey = []
        path_to_pred = []
        curr_dist_to_pred = len(BFS(agent_pos, predator.pos, g.graph))
        curr_dist_to_prey = len(BFS(agent_pos, prey.pos, g.graph))
        all_possible_moves = [(curr_dist_to_prey, curr_dist_to_pred, agent_pos)] #keeping list of all possible moves available to agent at any time including staying in place

        
        for neighbour in neighbours:
            path_to_prey = BFS(neighbour, prey.pos, g.graph)
            path_to_pred = BFS(neighbour, predator.pos, g.graph)

            #keeping all moves handy
            tup2 = (len(path_to_prey), len(path_to_pred), neighbour)
            all_possible_moves.append(tup2) 
            
            #making list of moves which only satisfy 1st condition of Agent 1
            if(len(path_to_pred) > curr_dist_to_pred and len(path_to_prey) < curr_dist_to_prey):
                tup3 = (len(path_to_prey), len(path_to_pred), neighbour)
                amazing_moves.append(tup3)                


        if(len(amazing_moves) > 0):         #if any moves satisfy agent 1s first condition    
            moves_sorted = sorted(amazing_moves, key=lambda t: (t[0], -t[1])) #sort by prey distance
            move_choice=[]
            for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]): 
                    move_choice.append(move)
            neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
            agent_pos = neighbour_choice[2]


        else: #dip into bag of all possible moves

            if (curr_dist_to_pred <= 3): #if predator too close then run away from predator
                moves_sorted = sorted(all_possible_moves, key=lambda t: (-t[1], t[0]))
                move_choice=[]
                for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                    if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]):
                        move_choice.append(move)
                neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
                agent_pos = neighbour_choice[2]

            else: #make a dash to the prey
                moves_sorted = sorted(all_possible_moves, key=lambda t: (t[0], -t[1]))
                move_choice=[]
                for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                    if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]):
                        move_choice.append(move)
                neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
                agent_pos = neighbour_choice[2]                

        path.append(agent_pos)

        steps += 1

        if (agent_pos == prey.pos): #if agent has already moved to location of prey
            break

        #move prey
        prey.prey_move(g)

        if (prey.pos == agent_pos): #if prey luckily moves to location of agent
            break
        
        #move predator
        predator.pred_move(g, agent_pos)

        if(predator.pos == agent_pos): #if predator catches agent
            fail = 1
            break
        
    #fail if timeout error    
    if(steps > 100):
        fail = 2     
    return fail, path

In [10]:
wins = 0
losses = 0
timeout = 0
times = 0
win_dist = []
loss_dist = []

while times < 3000:
    g = Graph()
    graph = g.addEdge()
    while graph:
        graph = g.addEdge()
    res, path = Agent2(g)
    
    if(res == 0):
        wins += 1
        win_dist.append(len(path))
    elif(res == 1):
        losses += 1
        loss_dist.append(len(path))
    else:
        timeout += 1
    times += 1
    
print('Number of wins: ' + str(wins))
print("Max Steps Taken During a Win: " + str(max(win_dist)-1))
print("Average Steps Taken During a Win: " + str((sum(win_dist)/wins)-1))
print('Number of losses: ' + str(losses))
print("Max Steps Taken During a Loss: " + str(max(loss_dist)-1))
print("Average Steps Taken During a Loss: " + str((sum(loss_dist)/losses)-1))
print('Number of timeouts: ' + str(timeout))
print("Success Rate: " + str((wins/times)))

Number of wins: 2988
Max Steps Taken During a Win: 64
Average Steps Taken During a Win: 10.262717536813922
Number of losses: 12
Max Steps Taken During a Loss: 22
Average Steps Taken During a Loss: 7.75
Number of timeouts: 0
Success Rate: 0.996
